In [18]:
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
linear_regression_model = LinearRegression()
ridge_regression_model = Ridge(alpha=1)
lasso_regression_model = Lasso(alpha=1)

In [19]:
# # calcularea valorii MSE și MAE
from sklearn.metrics import mean_squared_error, mean_absolute_error
# mse_value = mean_squared_error(y_true, y_pred)
# mae_value = mean_absolute_error(y_true, y_pred)

In [20]:
import numpy as np
from sklearn.utils import shuffle
# load training data
training_data = np.load('data/training_data.npy')
prices = np.load('data/prices.npy')
# print the first 4 samples
print('The first 4 samples are:\n ', training_data[:4])
print('The first 4 prices are:\n ', prices[:4])
# shuffle
training_data, prices = shuffle(training_data, prices, random_state=0)

The first 4 samples are:
  [[2.0150e+03 4.1000e+04 1.9670e+01 1.5820e+03 1.2620e+02 5.0000e+00
  1.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  1.0000e+00 0.0000e+00]
 [2.0110e+03 4.6000e+04 1.8200e+01 1.1990e+03 8.8700e+01 5.0000e+00
  1.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00
  1.0000e+00 0.0000e+00]
 [2.0120e+03 8.7000e+04 2.0770e+01 1.2480e+03 8.8760e+01 7.0000e+00
  1.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  1.0000e+00 0.0000e+00]
 [2.0130e+03 8.6999e+04 2.3080e+01 1.4610e+03 6.3100e+01 5.0000e+00
  1.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  1.0000e+00 0.0000e+00]]
The first 4 prices are:
  [12.5  4.5  6.   3.5]


In [21]:
def normalize_data(training_data, testing_data, type= None):
    scaler = StandardScaler()
    scaler.fit(training_data)
    scaler_train_data = scaler.transform(training_data)
    
    if testing_data is None:
        return scaler_train_data
    
    scaler_test_data = scaler.transform(testing_data)
    return scaler_train_data, scaler_test_data

In [22]:
num_samples_per_fold = len(training_data) // 3

training_data1, prices1 = training_data[:num_samples_per_fold], prices[:num_samples_per_fold]

training_data2, prices2 = training_data[num_samples_per_fold: 2 * num_samples_per_fold], prices[num_samples_per_fold:2*num_samples_per_fold]

training_data3, prices3 = training_data[2*num_samples_per_fold:], prices[2*num_samples_per_fold:]

In [23]:
def train_test(x_train, y_train, x_test, y_test):
    scaled_train_data, scaled_test_data = normalize_data(x_train, x_test)
    model = LinearRegression()
    model.fit(scaled_train_data, y_train)
    y_pred = model.predict(x_test)
    mse = mean_squared_error(y_pred, y_test)
    mae = mean_absolute_error(y_pred, y_test)
    return mse, mae

In [29]:
mse_1, mae_1 = train_test(np.concatenate((training_data1, training_data2)), np.concatenate((prices1, prices2)), training_data3, prices3)
mse_2, mae_2 = train_test(np.concatenate((training_data1, training_data3)), np.concatenate((prices1, prices3)), training_data2, prices2)
mse_3, mae_3 = train_test(np.concatenate((training_data2, training_data3)), np.concatenate((prices2, prices3)), training_data1, prices1)

In [30]:
print("Mean MS =", (mse_1 + mse_2 + mse_3)/3)

Mean MS = 65316421.333333336


In [31]:
print("Mean MS =", (mae_1 + mae_2 + mae_3)/3)

Mean MS = 5505.872395833333


In [32]:
def train_test_ridge(x_train, y_train, x_test, y_test, alpha=1):
    scaled_train_data, scaled_test_data = normalize_data(x_train, x_test, True)
    model = Ridge(alpha)
    model.fit(scaled_train_data, y_train)
    y_pred = model.predict(scaled_test_data)
    mse = mean_squared_error(y_pred, y_test)
    mae = mean_absolute_error(y_pred, y_test)
    return mse, mae

for alpha in [1, 10, 100, 100]:
    mse_1_ridge, mae_1_ridge = train_test_ridge(np.concatenate((training_data1, training_data2)), np.concatenate((prices1, prices2)), training_data3, prices3, alpha)
    mse_2_ridge, mae_2_ridge = train_test_ridge(np.concatenate((training_data1, training_data3)), np.concatenate((prices1, prices3)), training_data2, prices2, alpha)
    mse_3_ridge, mae_3_ridge = train_test_ridge(np.concatenate((training_data2, training_data3)), np.concatenate((prices2, prices3)), training_data1, prices1, alpha)
    print(mse_1_ridge, mae_1_ridge)
    print(mse_2_ridge, mae_2_ridge)
    print(mse_3_ridge, mae_3_ridge)
    print()

3.1752996 1.3207695
3.2212272 1.331149
3.1058664 1.3068019

3.1752465 1.3206646
3.2218485 1.3309039
3.1048813 1.3065406

3.179631 1.3206303
3.2335649 1.3299665
3.10373 1.3051705

3.179631 1.3206303
3.2335649 1.3299665
3.10373 1.3051705

